<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/ToolOrchestra_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- 1. IMPORTS & API SETUP ---
import google.genai as genai
from google.genai import types
import os
from typing import Dict, Any, Tuple

# --- Secure API Client Initialization ---
GEMINI_API_KEY = None
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# *** ALL LLM ROLES WILL USE THIS MODEL ID ***
REQUESTED_MODEL_ID = 'gemini-3-pro-preview'
client = None

if GEMINI_API_KEY:
    try:
        client = genai.Client(api_key=GEMINI_API_KEY)
        print(f"✅ Gemini client configured for **{REQUESTED_MODEL_ID}**.")
    except Exception as e:
        print(f"❌ Client initialization failed: {e}")
else:
    print("❌ API Key not found. Please ensure your key is set up.")


# --- 2. AGENT CONFIGURATIONS (Using gemini-3-pro's thinking_level) ---
# NOTE: thinking_level is ONLY supported by Gemini 3 models (gemini-3-pro-preview).

def get_low_think_config():
    """Lowest reasoning effort: fastest, lowest cost, suitable for Generalist/Router."""
    return types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_level="low"
        )
    )

def get_high_think_config():
    """Highest reasoning effort: higher quality, higher latency/cost, suitable for Specialist/Base LLM."""
    return types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            thinking_level="high",
            include_thoughts=False # Optional: set to True to inspect internal reasoning
        )
    )

def get_standard_config():
    """Standard config for tools or models that don't use thinking_level."""
    return types.GenerateContentConfig()

# --- 3. CORE TOOLORCHESTRA CLASSES (All LLMs use the same model, different configs/costs) ---

class BaseLLM:
    """The 'Deep Think' version: High Accuracy, Very High Cost (highest thinking level)."""
    NAME = "Base_LLM"
    MODEL_ID = REQUESTED_MODEL_ID
    COST = 0.15
    ACCURACY = 0.99

    def __init__(self, client: genai.Client):
        self.client = client
        self.config = get_high_think_config()

    def execute(self, task: str) -> str:
        if not self.client:
            return f"[SIMULATED] Base LLM processed '{task}' (HIGH Think)."

        print(f"    ... Calling {self.MODEL_ID} with **HIGH Thinking** (Highest Cost).")
        try:
            response = self.client.models.generate_content(
                model=self.MODEL_ID,
                contents=task,
                config=self.config
            )
            return response.text
        except Exception as e:
            return f"[ERROR] Base LLM execution failed: {e}"


class SpecialistModel:
    """The 'Focused Think' version: High Accuracy, Medium Cost (HIGH thinking for targeted tasks)."""
    NAME = "Specialist_LLM"
    MODEL_ID = REQUESTED_MODEL_ID
    COST = 0.08 # Lower cost than Base, higher than Generalist
    ACCURACY = 0.95

    def __init__(self, client: genai.Client):
        self.client = client
        self.config = get_high_think_config() # Still using HIGH thinking for specialization

    def execute(self, task: str) -> str:
        if not self.client:
            return f"[SIMULATED] Specialist solved '{task}' (HIGH Think)."

        print(f"    ... Calling {self.MODEL_ID} with **HIGH Thinking** (Medium Cost).")
        try:
            response = self.client.models.generate_content(
                model=self.MODEL_ID,
                contents=f"You are a specialized code expert. Only provide the code, no explanation. Task: {task}",
                config=self.config
            )
            return response.text
        except Exception as e:
            return f"[ERROR] Specialist LLM execution failed: {e}"


class GeneralistModel:
    """The 'Router' version: Medium Accuracy, Low Cost (LOW thinking level)."""
    NAME = "Generalist_LLM"
    MODEL_ID = REQUESTED_MODEL_ID
    COST = 0.02 # Lowest cost LLM option
    ACCURACY = 0.80

    def __init__(self, client: genai.Client):
        self.client = client
        self.config = get_low_think_config() # Use LOW thinking for speed/low cost

    def execute(self, task: str) -> str:
        if not self.client:
            return f"[SIMULATED] Generalist provided an answer for '{task}' (LOW Think)."

        print(f"    ... Calling {self.MODEL_ID} with **LOW Thinking** (Lowest Cost).")
        try:
            response = self.client.models.generate_content(
                model=self.MODEL_ID,
                contents=f"Provide a concise, single-paragraph answer to the following: {task}",
                config=self.config
            )
            return response.text
        except Exception as e:
            return f"[ERROR] Generalist LLM execution failed: {e}"


class CalculatorTool:
    """The Non-LLM Tool: Perfect Accuracy, Negligible Cost."""
    NAME = "Calculator_Tool"
    COST = 0.001
    ACCURACY = 1.0
    def __init__(self, client: genai.Client):
        pass

    def execute(self, task: str) -> str:
        return f"[Tool Used] Tool calculated the result for '{task}' precisely. (Actual computation skipped for demo.)"

# --- 4. THE TOOLORCHESTRATOR CLASS ---

class ToolOrchestrator:
    """Dynamically selects the best resource based on task type and optimization goal."""
    def __init__(self, client: genai.Client, optimization_goal: str = "smart_balance"):
        self.resources: Dict[str, Any] = {
            "Base_LLM": BaseLLM(client),
            "Specialist_LLM": SpecialistModel(client),
            "Generalist_LLM": GeneralistModel(client),
            "Calculator_Tool": CalculatorTool(client)
        }
        self.knowledge_base: Dict[str, str] = {
            "math_query": "Calculator_Tool",
            "coding_task": "Specialist_LLM",
            "complex_analysis": "Base_LLM",
            "general_info": "Generalist_LLM"
        }
        # Weights for Smart Balance (Acc/Cost)
        self.weights: Dict[str, Tuple[float, float]] = {
            "high_accuracy": (0.8, 0.2),
            "low_cost": (0.2, 0.8),
            "smart_balance": (0.5, 0.5)
        }
        self.optimization_goal = optimization_goal

    def classify_task(self, task: str) -> str:
        """SLM-based routing function."""
        task_lower = task.lower()
        if any(keyword in task_lower for keyword in ["calculate", "multiply", "divide", "add"]):
            return "math_query"
        elif any(keyword in task_lower for keyword in ["code", "algorithm", "python", "javascript"]):
            return "coding_task"
        elif any(keyword in task_lower for keyword in ["analyze", "predict", "synthesize", "strategy"]):
            return "complex_analysis"
        else:
            return "general_info"

    def calculate_score(self, resource_instance: Any, goal: str) -> float:
        """Calculates a weighted score (Smart Balance)."""
        acc_weight, cost_weight = self.weights.get(goal, self.weights["smart_balance"])

        # Invert cost so lower cost contributes to a higher score
        score = (resource_instance.ACCURACY * acc_weight * 10) + \
                ((1.0 - resource_instance.COST * 5) * cost_weight)

        return score

    def orchestrate(self, task: str, goal: str = None) -> str:
        if goal is None:
            goal = self.optimization_goal

        task_type = self.classify_task(task)

        highest_score = -float('inf')
        final_resource = None

        # Define which resources are relevant for the task type
        if task_type == "math_query":
            relevant_resources = [self.resources["Calculator_Tool"], self.resources["Base_LLM"]]
        elif task_type == "coding_task":
            relevant_resources = [self.resources["Specialist_LLM"], self.resources["Base_LLM"]]
        elif task_type == "complex_analysis":
            relevant_resources = [self.resources["Base_LLM"], self.resources["Specialist_LLM"], self.resources["Generalist_LLM"]]
        else: # general_info
            relevant_resources = [self.resources["Generalist_LLM"], self.resources["Base_LLM"]]

        # Check scores against the goal to perform dynamic routing
        for resource in relevant_resources:
            score = self.calculate_score(resource, goal)
            if score > highest_score:
                highest_score = score
                final_resource = resource

        if final_resource is None:
            final_resource = self.resources.get(self.knowledge_base.get(task_type, "Generalist_LLM")) # Fallback

        print("-" * 70)
        print(f"**Task:** {task}")
        print(f"**Type:** {task_type.replace('_', ' ').title()}")
        print(f"**Goal:** {goal.replace('_', ' ').title()}")
        print(f"**Selected Resource (Dynamic Routing):** {final_resource.NAME} (Cost: ${final_resource.COST:.3f}, Score: {highest_score:.2f})")

        # Execute the task
        result = final_resource.execute(task)
        return f"\n--- Resource Output ---\n{result}"

# --- 5. DEMONSTRATION EXECUTION ---

if client and GEMINI_API_KEY:
    try:
        orchestrator = ToolOrchestrator(client=client, optimization_goal="smart_balance")

        # 1. Complex Analysis (High Accuracy): Routes to Base_LLM (HIGH Think)
        print("\n=== DEMO 1: Complex Analysis, HIGH ACCURACY GOAL ===")
        print(orchestrator.orchestrate(
            "Synthesize a five-point strategy for entering the European AI market in 2026, analyzing regulatory hurdles.",
            goal="high_accuracy"
        ))

        # 2. Complex Analysis (Low Cost): Routes to Generalist_LLM (LOW Think)
        print("\n=== DEMO 2: Complex Analysis, LOW COST GOAL ===")
        print(orchestrator.orchestrate(
            "Synthesize a five-point strategy for entering the European AI market in 2026, analyzing regulatory hurdles.",
            goal="low_cost"
        ))

        # 3. Coding Task (Smart Balance): Routes to Specialist_LLM (HIGH Think for code quality)
        print("\n=== DEMO 3: Coding Task, SMART BALANCE GOAL ===")
        print(orchestrator.orchestrate(
            "Write a simple Python function to calculate the Fibonacci sequence up to N.",
            goal="smart_balance"
        ))

    except Exception as e:
        print(f"\n[CRITICAL ERROR] Failed to run orchestration demo: {e}")
        print("\nEnsure your API key is valid and you have access to the requested model.")
else:
    print("\nCannot run demo: Client was not initialized successfully.")

✅ Gemini client configured for **gemini-3-pro-preview**.

=== DEMO 1: Complex Analysis, HIGH ACCURACY GOAL ===
----------------------------------------------------------------------
**Task:** Synthesize a five-point strategy for entering the European AI market in 2026, analyzing regulatory hurdles.
**Type:** Complex Analysis
**Goal:** High Accuracy
**Selected Resource (Dynamic Routing):** Base_LLM (Cost: $0.150, Score: 7.97)
    ... Calling gemini-3-pro-preview with **HIGH Thinking** (Highest Cost).

--- Resource Output ---
Entering the European AI market in 2026 requires navigating a landscape defined by the fully implemented **EU AI Act**, strict enforcement of **GDPR**, and the emerging **AI Liability Directive**. By 2026, the transition periods for the AI Act will have largely expired, moving the region from a "regulatory warning" phase to active enforcement.

Here is a synthesized five-point strategy for market entry, pairing strategic actions with the specific regulatory hurdles 